# Testing the TiGL Geometry Library with Jupyter/BeakerX Java kernel

— Agostino De Marco, University of Naples Federico II

This Jupyter notebook uses the [BeakerX Java kernel](http://beakerx.com). The purpose is demonstrating the use of Java and of external native libraries in the context of a notebook. The library used here is [TiGL](https://github.com/DLR-SC/tigl/), a C++ open source library by the [German aerospace research centre DLR](https://software.dlr.de/p/tigl/home/), providing functions to process aircraft geometries in pre-design. TiGL is developed around the [CPACS input/output format](https://software.dlr.de/p/cpacs/home/). CPACS stands for Common Parametric Aircraft Configuration Schema (CPACS) and enables engineers to define an air transportation system in a way which is suitable for multi-disciplinary and multi-fidelity design in distributed environments.

## Enable TiGL bindings for Java

TiGL is written in C++ and depends on several other libraries, such as [TIXI](https://software.dlr.de/p/tixi/home/) for processing XML files and [Open CASCADE Technology (OCCT)](https://www.opencascade.com/content/overview) for implementing aircraft oriented CAD functionalities.

[TiGL distributions](https://github.com/DLR-SC/tigl/releases) include software bindings for Java, which are made available by pointing the JVM to necessary JAR files as well as to their companion native library files. For simplicity, all the extra file needed by this example have been copied in the same folder where the notebook lives.

BeakerX offers the `%classpath` magic to add jars to the kernel. We use this command in the next cell to make the `de.dlr.sc.tigl3.*` classes available.

In [1]:
%classpath add jar ./tigl-3.0.0rc1.jar
%classpath add jar ./jna-4.5.1.jar
%classpath add jar ./jna-platform-4.5.1.jar
%classpath

Added jar: [tigl-3.0.0rc1.jar]
Added jar: [jna-4.5.1.jar]
Added jar: [jna-platform-4.5.1.jar]
C:\Users\agodemar\AppData\Local\Temp\beaker4312695700357892654\outDir
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\tigl-3.0.0rc1.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jna-4.5.1.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jna-platform-4.5.1.jar


(Get the above JARs from [TiGL current release page](https://github.com/DLR-SC/tigl/releases) and [JNA library page](https://github.com/java-native-access/jna))

Normally the Java `import` statement only works in the cell where you use it. To make a class automatically imported into all cells, we use the `%import` magic. So, the following cell makes the classes `Tigl`, `TiglNativeInterface`, and `CpacsConfiguration` visible to all the subsequent cells.

In [2]:
%import de.dlr.sc.tigl3.Tigl
%import de.dlr.sc.tigl3.TiglNativeInterface
%import de.dlr.sc.tigl3.CpacsConfiguration

(No semicolon `;` at the end)

In this example we are going to use Windows 64-bits, therefore the following files are copied into the notebook folder: `tigl3.lib`, `tigl3.dll`, `tixi3.dll`, `concrt140.dll`, `msvcp140.dll`, `vcomp140.dll`, `vcruntime140.dll`.

Since we have Java, let's double check:

In [3]:
import java.nio.file.*;
Files.list(Paths.get(".")) // a stream of paths
    .map(path -> path.toString().toLowerCase())
    .filter(name -> name.matches("([^\\s]+(\\.(?i)(lib|dll))$)")) // use regex
    .forEach(System.out::println);

.\concrt140.dll
.\msvcp140.dll
.\tigl3.dll
.\tigl3.lib
.\tixi3.dll
.\vcomp140.dll
.\vcruntime140.dll


null

## Get the TiGL version

In [4]:
System.out.println("TiGL Version: " + Tigl.getVersion());

TiGL Version: 3.0.0rc1-rc0552217


null

## A first example: Read a CPACS file and look for profiles

We have an aircraft geometry file `CPACS_30_D150.xml` modelled in CPACS (v3.0) format. Let's read the file and explore some of its data structures.

First, we need to package the algorithm into a class to make it usable via an instance. We'll call this class `Test1`. We'll pass the file name to the constructor and successively invoke the method `Test1.run()`.

In [5]:
package test.tigl;

import java.util.List;
import de.dlr.sc.tigl3.TiglBSpline;
import de.dlr.sc.tigl3.TiglPoint;
import de.dlr.sc.tigl3.TiglException;

public class Test1 {
    
    public String fileName;
    public List<TiglBSpline> splines;
    
    public void run(String fileName) {
        this.fileName = fileName;
        // read the CPACS file
        try (CpacsConfiguration config = Tigl.openCPACSConfiguration(fileName, "")) {
            // get splines from a profile NACA0012
            System.out.println("Get splines from profile NACA0012:");
            this.splines = config.getProfileSplines("NACA0012");
            System.out.println("Splines found: " + splines.size());
            System.out.println("---");
            for (int ispl = 0; ispl < splines.size(); ++ispl) {
                System.out.println("Spline: " + ispl);
                TiglBSpline spl = splines.get(ispl);
                System.out.println("Degree:" + spl.degree + ", Ctrl Pts: " + spl.controlPoints.size() + ", Knots: " + spl.knots.size());
            }
        } catch(TiglException err) {
            System.out.println(err.getMessage());
            System.out.println(err.getErrorCode());
        }
    }
}

test.tigl.Test1

Now let's instantiate a `Test1` object and use it:

In [6]:
package test.tigl;

import com.twosigma.beakerx.chart.xychart.*;
import java.util.List;
import java.util.stream.Collectors;

String fileName = "CPACS_30_D150.xml";
Test1 test1 = new Test1();
test1.run(fileName);
List<Object> xs = test1.splines.get(0).controlPoints.stream()
    .map(pt -> pt.getX()).collect(Collectors.toList());
List<Number> ys = test1.splines.get(0).controlPoints.stream()
    .map(pt -> pt.getZ()).collect(Collectors.toList());

test1.splines.get(0).controlPoints.stream()
    //.forEach(pt -> System.out.println(String.format("%f , %f", pt.getX(), pt.getZ())));
    .forEach(pt -> System.out.println(pt));

Plot plot = new Plot();
plot.setXBound(0.0, 1.0).setYBound(-0.5, 0.5);
plot.setTitle(String.format("File: %s - NACA0012 Splines", fileName));
plot.setXLabel("x/c").setYLabel("y/c");
Line line1 = new Line(xs, ys);
plot.add(line1);
plot.display();

feb 03, 2018 11:32:58 PM de.dlr.sc.tigl3.Tigl openCPACSConfiguration
INFORMAZIONI: TiGL: Cpacs configuration opened succesfully


Get splines from profile NACA0012:
Splines found: 2
---
Spline: 0
Degree:3, Ctrl Pts: 82, Knots: 86
Spline: 1
Degree:3, Ctrl Pts: 83, Knots: 87


feb 03, 2018 11:32:58 PM de.dlr.sc.tigl3.CpacsConfiguration closeDocument
INFORMAZIONI: Closing cpacs configuration 


(0.9875, 0.9875, 0.9875)
(0.9833339221696136, 0.9833339221696136, 0.9833339221696136)
(0.9750017226786822, 0.9750017226786822, 0.9750017226786822)
(0.9625016562185684, 0.9625016562185684, 0.9625016562185684)
(0.9500015946500225, 0.9500015946500225, 0.9500015946500225)
(0.9375015371140465, 0.9375015371140465, 0.9375015371140465)
(0.9250014836306389, 0.9250014836306389, 0.9250014836306389)
(0.9125014339935235, 0.9125014339935235, 0.9125014339935235)
(0.9000013880654013, 0.9000013880654013, 0.9000013880654013)
(0.8875013456984296, 0.8875013456984296, 0.8875013456984296)
(0.875001306754715, 0.875001306754715, 0.875001306754715)
(0.862501271100214, 0.862501271100214, 0.862501271100214)
(0.8500012386055699, 0.8500012386055699, 0.8500012386055699)
(0.8375012091452574, 0.8375012091452574, 0.8375012091452574)
(0.8250011825969357, 0.8250011825969357, 0.8250011825969357)
(0.8125011588408556, 0.8125011588408556, 0.8125011588408556)
(0.8000011377591977, 0.8000011377591977, 0.8000011377591977)
(0.78

null